In [ ]:
import opt_note.scsp as scsp
import hexaly.optimizer

In [ ]:
import marimo as mo
import nbformat

# WMM_HEXALY モデルを max 演算で考えてみる

元の `WMM_HEXALY` モデルは `WMM` をパラメータ化しただけだったため,
一部のパラメータを動かしただけでは解が更新されにくかった. Majority Merge の部分を「最も大きい Weight を持つ文字を採用する」にすれば Weight の変換に解が追従しやすくなるのではないか.

In [ ]:
class Model:
    def __init__(
        self, instance: list[str], initial_weights: list[list[int]] | None = None
    ):
        chars = "".join(sorted(list(set("".join(instance)))))

        hxoptimizer = hexaly.optimizer.HexalyOptimizer()
        hxmodel = hxoptimizer.model
        assert isinstance(hxoptimizer, hexaly.optimizer.HexalyOptimizer)
        assert isinstance(hxmodel, hexaly.optimizer.HxModel)

        # 重みの最大値は初期重みが与えられた場合は初期重みの最大値の 2 倍,
        # 初期重みが与えられなかった場合は文字種数とする.
        max_weight = (
            max(max(w, len(s)) for s, ws in zip(instance, initial_weights) for w in ws)
            if initial_weights
            else len(chars)
        )
        priorities1d = [
            hxmodel.int(1, max_weight) for s in instance for cidx, _ in enumerate(s)
        ]

        func = hxmodel.create_int_external_function(self.objective)
        func.external_context.lower_bound = 0
        func.external_context.upper_bound = sum(len(s) for s in instance)

        indices_1d_to_2d: list[tuple[int, int]] = []
        counter = 0
        for s in instance:
            indices_1d_to_2d.append((counter, counter + len(s)))
            counter += len(s)

        self.instance = instance
        self.chars = chars
        self.hxoptimizer = hxoptimizer
        self.hxmodel = hxmodel
        self.priorities1d = priorities1d
        self.indices_1d_to_2d = indices_1d_to_2d

        # これらが実行される時点で self.* が必要になるため初期化の最後に移動

        hxmodel.minimize(func(*priorities1d))
        hxmodel.close()

        if initial_weights:
            priorities2d = self.priorities_1d_to_2d(priorities1d)
            for ps, ws in zip(priorities2d, initial_weights):
                for p, w in zip(ps, ws):
                    p.set_value(w)

    def solve(self, time_limit: int | None = 60, log: bool = False) -> "Model":
        assert isinstance(self.hxoptimizer.param, hexaly.optimizer.HxParam)
        if time_limit is not None:
            self.hxoptimizer.param.time_limit = time_limit
        self.hxoptimizer.param.verbosity = 1 if log else 0
        self.hxoptimizer.solve()
        return self

    def to_solution(self) -> str | None:
        assert isinstance(self.hxoptimizer.solution, hexaly.optimizer.HxSolution)
        status = self.hxoptimizer.solution.status
        if status not in {
            hexaly.optimizer.HxSolutionStatus.OPTIMAL,
            hexaly.optimizer.HxSolutionStatus.FEASIBLE,
        }:
            return None

        priorities1d_value: list[int] = [p.value for p in self.priorities1d]
        priorities2d_value = self.priorities_1d_to_2d(priorities1d_value)
        return self.wmm(priorities2d_value)

    def wmm(self, priorities2d: list[list[int]]) -> str:
        indices = tuple(0 for _ in self.instance)
        solution = ""

        # while not all(idx == len(s) for idx, s in zip(indices, self.instance)):
        for _ in range(len(self.instance) * max(len(s) for s in self.instance)):
            if all(idx == len(s) for idx, s in zip(indices, self.instance)):
                break

            counts = [
                max(
                    [0] + [
                        priorities2d[sidx][idx]
                        for sidx, (idx, s) in enumerate(zip(indices, self.instance))
                        if idx < len(s) and s[idx] == c
                    ]
                )
                for c in self.chars
            ]
            next_char = self.chars[counts.index(max(counts))]

            solution += next_char
            indices = tuple(
                idx + 1 if idx < len(s) and s[idx] == next_char else idx
                for idx, s in zip(indices, self.instance)
            )

        return solution

    def priorities_1d_to_2d[T](self, priorities1d: list[T]) -> list[list[T]]:
        return [priorities1d[start:end] for start, end in self.indices_1d_to_2d]

    def objective(self, priorities1d: list[int]) -> int:
        priorities2d = self.priorities_1d_to_2d(
            [priorities1d[i] for i in range(len(priorities1d))]
        )
        solution = self.wmm(priorities2d)
        return len(solution)

In [ ]:
def solve(
    instance: list[str], time_limit: int | None = 60, log: bool = False
) -> str | None:
    return Model(instance).solve(time_limit, log).to_solution()

In [ ]:
def bench(instance: list[str]) -> None:
    model = Model(instance).solve()
    solution = model.to_solution()
    scsp.util.show(instance)
    if solution is not None:
        scsp.util.show(instance, solution)
        print(f"solution is feasible: {scsp.util.is_feasible(instance, solution)}")
    else:
        print("--- Solution not found ---\n")

    print(f"solution is optimal: {model.hxoptimizer.solution.status == hexaly.optimizer.HxSolutionStatus.OPTIMAL}")
    print(f"bset bound: {model.hxoptimizer.solution.get_objective_bound(0)}")

In [ ]:
def bench0(instance: list[str]) -> None:
    model = scsp.model.wmm_hexaly.Model(instance).solve()
    solution = model.to_solution()
    scsp.util.show(instance)
    if solution is not None:
        scsp.util.show(instance, solution)
        print(f"solution is feasible: {scsp.util.is_feasible(instance, solution)}")
    else:
        print("--- Solution not found ---\n")

    print(f"solution is optimal: {model.hxoptimizer.solution.status == hexaly.optimizer.HxSolutionStatus.OPTIMAL}")
    print(f"bset bound: {model.hxoptimizer.solution.get_objective_bound(0)}")

In [ ]:
bench(scsp.example.load("uniform_q26n004k015-025.txt"))

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 62) ---
 Sol: ultcikgenykcosouhmvajiqfozpxplnhtqgbxzrddxbcvxsuqpvissbxrvnngf
str1: --t--kg-n-k----uhm--------px--nhtqg-xz------vx-----is---------
str2: ----i-------o-------jiqfo----ln----bx----x-cv-suqpvissbx-----f
str3: ul-ci---ny-coso---v-----ozp-pl-------------------p------------
str4: ----i-ge----------va-----z--------gb--rdd-bc--s---v-----rvnngf

solution is feasible: True
solution is optimal: False
bset bound: 0


In [ ]:
bench0(scsp.example.load("uniform_q26n004k015-025.txt"))

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 62) ---
 Sol: tkulcigneycojiqfsovoazpkuhmplxnhtqgbxzrddbxcvsxuqpvissbxrvnngf
str1: tk----gn---------------kuhmp-xnhtqg-xz------v-x----is---------
str2: -----i-----ojiqf-o----------l-n----bx-----xcvs-uqpvissbx-----f
str3: --ulci-n-yco----sovo-zp----pl--------------------p------------
str4: -----ig-e---------v-az------------gb--rddb-c-s----v-----rvnngf

solution is feasible: True
solution is optimal: False
bset bound: 0


In [ ]:
bench(scsp.example.load("uniform_q26n008k015-025.txt"))

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 105) ---
 Sol: ipbdtkgevnkulahmzgbrdcvinyzdbcopfjtisvxnhtqfolgervonbnwxxczvxsuqckpmsbroqvigtdfprlsuivctodtmpesbobhrpxwfd
str1: ----tkg--nku--hm---------------p------xnhtq---g--------x--zvx-------------i-------s----------------------
str2: i-----------------------------o--j-i------qfol-----nb--xxc-v-suq--p------vi-------s-----------sb-----x-f-
str3: -----------ul--------c-iny---co-----s-------o----vo-------z-------p------------p-l----------p------------
str4: i-----gev----a--zgbrd------dbc------sv----------rv-n-n---------------------g--f--------------------------
str5: -p-----------------------y-----p-------------l--r---------z-x-u-c-pm----qv-gtdf----uivc--d----sbo--------
str6: -pbd---ev-----------dcv

In [ ]:
bench0(scsp.example.load("uniform_q26n008k015-025.txt"))

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 102) ---
 Sol: ipuyplrogenbdcevadzfxwgbjtinyucvxerdpmqfozkrdbrlcsovgtondfkuibhtmpxrvnhntqgxzpcvxwdsubroqpvisslbxfbhep
str1: -------------------------t----------------k---------g--n--ku--h-mpx--nh-tqgxz--vx----------is---------
str2: i------o----------------j-i-----------qfo------l-------n-----b----x--------x--cv---su---qpviss-bxf----
str3: --u--l-------c------------iny-c---------o--------sov--o---------------------zp-----------p----l------p
str4: i-------ge-----va-z---gb----------rd--------db--cs-v---------------rvn-n--g----------------------f----
str5: -p-yplr-----------z-x--------uc-----pmq------------vgt--df-ui-------v---------c---ds-b-o--------------
str6: -p---------bd-ev-d------------cv---dp--f-

In [ ]:
bench(scsp.example.load("uniform_q26n016k015-025.txt"))

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 152) ---
  Sol: ikrshtxxuokqjnafpbadigqfejwolkcinbycokplrzxuxsfjhcpafqkrdbicgzpmpxqvgnctbvxerdqwazhckotvodpfzsmiuiqgspbrviddbgplecsvrwtaodqvsbbhnntfmprgqpxzvxisjfuowwyd
str01: -----t----k----------g----------n----k-----u----h--------------mpx---n------------h---t-----------qg--------------------------------------xzvxis--------
str02: i--------o--j-------i-qf---ol---nb--------x-x----c-----------------v-------------------------s--u-q--p--vi--------s---------sb------------x------f------
s

In [ ]:
bench0(scsp.example.load("uniform_q26n016k015-025.txt"))

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 156) ---
  Sol: irkepxulcswxqkarifsnydhtxpabcuojivncdqgbevzfakolrzgnkbxqujtpvxerdzqcevdwhbcsvrvwantfigplmqvgctdijowfuqpvokiszpsbxnhamiuvdtqgcdbfxezhknsbkroqvbbhmprlpgxwdyis
str01: -----------------------t---------------------k----gnk---u---------------h---------------m-------------p---------xnh------tqg----x-z---------v---------x---is
str02: i-----------------------------oji----q-----f--ol---n-bx------x-----c-v-----s------------------------uqpv--is--sbx--------------f------------------

In [ ]:
bench(scsp.example.load("uniform_q05n010k010-010.txt"))

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 28) ---
  Sol: baedcbacdbeeecdabdcebadeaced
str01: ---dcb-c-----cd-b-c------ce-
str02: b--d----dbeee------eb-d-----
str03: ----c-acd-ee-c-----eb--e----
str04: -aed----d-----d--d-eb-d----d
str05: -a--cb----ee-c-ab-ce--------
str06: b----ba--be-----bdc-ba------
str07: b----ba---e----a---ebad-a---
str08: --e-------eeec--bd--b--e--e-
str09: ----c--cd-ee--da-dc---d-----
str10: b--d--a--b----d-b--e-a--a--d

solution is feasible: True
solution is optimal: False
bset bound: 0


In [ ]:
bench0(scsp.example.load("uniform_q05n010k010-010.txt"))

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 27) ---
  Sol: bbaedcdbeacdeecbdbeacbdacde
str01: ----dc-b--c---c-db--c---c-e
str02: b---d-dbe---ee----e--bd----
str03: -----c---acdeec---e--b----e
str04: --aed-d----d----d-e--bd--d-
str05: --a--c-be---e-c----a-b--c-e
str06: bba----be------bd---cb-a---
str07: bbae-----a--e--b---a--da---
str08: ---e----e---eecbdbe-------e
str09: -----c----cdee--d--a--d-cd-
str10: b---d----a-----bdbea---a-d-

solution is feasible: True
solution is optimal: False
bset bound: 0


In [ ]:
bench(scsp.example.load("uniform_q05n050k010-010.txt"))

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution (of length 36) ---
  Sol: adcebdabcedabceadbcaebdceba

In [ ]:
bench0(scsp.example.load("uniform_q05n050k010-010.txt"))

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution (of length 34) ---
  Sol: daecbdeabecdabcedaecbdaebcd

In [ ]:
bench(scsp.example.load("nucleotide_n010k010.txt"))

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 24) ---
  Sol: ATGGTGACCTACGAATCTGATACC
str01: ATGG-GA--TACG-----------
str02: AT----ACCT-----TC-----CC
str03: -------C--ACGAAT-TGA----
str04: -T----A---A--AATCTG-T---
str05: A-GGT-A---AC-AA----A----
str06: -T--T--CCTA-G-----G-TA--
str07: -T--TG---TA-GA-TCT------
str08: -TGG-GA---A-G--T-T----C-
str09: -T--T--CC-AC-AA-CT------
str10: -T-----C-TA--AA-C-GA-A--

solution is feasible: True
solution is optimal: False
bset bound: 0


In [ ]:
bench0(scsp.example.load("nucleotide_n010k010.txt"))

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 24) ---
  Sol: TCATGACCGTAGATCAGTACGATC
str01: --ATG---G--GAT-A---CG---
str02: --AT-ACC-T---TC----C---C
str03: -CA---C-G-A-AT---T--GA--
str04: T-A--A----A-ATC--T--G-T-
str05: --A-G---GTA-A-CA--A--A--
str06: T--T--CC-TAG----GTA-----
str07: T--TG----TAGATC--T------
str08: T---G---G--GA--AGT----TC
str09: T--T--CC--A---CA--AC--T-
str10: TC-T-A----A-A-C-G-A--A--

solution is feasible: True
solution is optimal: False
bset bound: 0


In [ ]:
bench(scsp.example.load("nucleotide_n050k050.txt"))

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

---
str09: A----C----T------C--G--------G--CT-G-----CA------TGCT--------T-A-----GT---G-----CACTC-A-----C----G--CAGTA----------TA----A----T---TA----A---T---A-AC---TA-A--T--TA----
str10: --T-------TGTAGATC---T-------G---T--T----C-------T-CTA---A-----A-C---G-A-A------C--T-----TTA---A---A-A-T--C--------T-G-TG-----TG-G--C-T---G-T-C-A--C---T----C---------
str11: -G---C-A---G-AG--C-A-T-T-T-------T-------C-------T---A---A---T-A------T-C-------CAC---AA---A---ATG-A-AG-----G-C-A---A--T-A---AT---T--GT-A-----CTA--C---T----C---------
str12: A-T---GA---G-----CCA--A------GA--T-------C--CGA----C---G-A-----A-----G-A--G-----C-C-CCAAG--------G-A--G-----G---A-G-A----AG-GA-G-G---G--A-----C----C-C------C--C------
str13: --T--C----T------C-A----C-A--G---T--T----CA---AGA----AC---C---CAA-A--GTAC-------C-C-CC------C--AT--A--G---C---CC---T--CT------T----A----AAG---C----CAC----------------
str14: AG----G---T-T---T--A-TA-C---C----T--T----C--C----T---A-GG----T-AACAA---AC-------CA----A-----C-CA---AC--T-----------T---

----GG---GT--C---CCACGT---TGA---A--------------------------------------R-
str27: A--AA-G----GT---T----TAT--A-C---CT--T----C--C------C-A-GG----T-AACAA---AC-------CA----A-----C-CA---AC--T-----------T---T---CGAT-C-T-C-T-----TG------------------------
str28: AGTA--G---T-T----C--G---C---C----T-GT-GTG-AGC----TG--AC--A-----AAC----T----TAGTA--------GT-------G-----T-----------T---TG-----TG---A-G----GAT--TA---------------------
str29: --T-------T-TA--T--A----C---C----T--T----C--C----T---A-GG----T-AACAA---AC-------CA----A-----C-CA---AC--T-----------T---T---CGAT-C-T-C-T-----TG-TAGA----T--------------
str30: A-T---G-C--G--G-TC--GT--CT--C----T-------C--C------C--CGG-C--T--------T----T--T----T-----TT-C-C-----C-----C-G-C---G---C----CG---CGT---T---G--GC--G-C-CG-A-------------
str31: -GT---GACA---A-A---A--A-C-A------TA--A-TG--G--A----CT-C---C----AACA-C---CA-T-GT-CA----A-G---CT--T------T--CAGG-----TAG---A-C------------------------------------------
str32: -GT---G---T--A-A----G-A---AAC-A----GTA----AGC----

bset bound: 0


In [ ]:
bench0(scsp.example.load("nucleotide_n050k050.txt"))

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

A-A---C---T--T-A--G-T-AGT--G--T---T--T--GT-GA---G---GAT---T-A-------------------
str29: -T---T----TA---TAC---C-T--T-C--C--T--A---G-----G-T--A---ACA-A--A-C----CA---A-C----C--A-A-CT--T---TCGATCT---C--T---T--GT-A--GA--T--------
str30: ATG-C-G--GT--CGT-C-T-C-TC---C--C---CG----G-C-T---T-----T-----T----T--T--TC---C----C-C---GC-G--C---CG--C-G-T---T--G---G-C---G-CC-GA------
str31: --G--TGAC--AA---A-A---A-CAT--A---AT-G----G--A-C--T-C--C-A-A---CA-C----CAT-G-TCA--A-GC-T---T--TCAG--G-T---A--GA-C------------------------
str32: --G--TG---TAA-G-A-A---A-CA-G--T--A---A---G-C--C----C-G-----GA--AG-T-G-----G-T--GT-----T---T--T--G-CGAT-T--TCGA---G---G-C--CG----G-------
str33: --GA--GA---A---T----G-A----G--TC--TC-AT---T-A-C----C-GC--C----C-G---GT-A-C--T---TA-GCA-AGCT-A--A-T--A---G-TC-A-C-G---G-C----------------
str34: ATG--TG--GT--CG-A--TGC--CATG----GA--G----G-C--C----CA-C--CAG-T----T---CAT---T-A--A-G----GCT---C---C--T--G---G--CA-T---T-----------------
str35: A---C-GA-G---CGT---T---T--T--A---A--G----G-----G

In [ ]:
bench(scsp.example.load("protein_n010k010.txt"))

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 45) ---
  Sol: MAEPQSSKLSNFDVEHYCAIPGVTRNQHAFGNELSRNPKGYTQHD
str01: MA------LS------YC--P-----------------KG-T---
str02: M---QSS-L-N-------AIP-V----------------------
str03: M--P----LS------Y---------QH-F-----R--K------
str04: M-E-----------EH------V--N------EL---------HD
str05: M----S----NFD-----AI----R---A----L-----------
str06: M----------F------------RNQ----N--SRN--G-----
str07: M----------F----Y-A--------HAFG--------GY----
str08: M----S-K---F-----------TR----------R-P--Y-Q--
str09: M----S-----F-V----A--GVT----A-------------Q--
str10: M-E--S--L----V------PG-------F-NE------------

solution is feasible: True
solution is optimal: False
bset bound: 0


In [ ]:
bench0(scsp.example.load("protein_n010k010.txt"))

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 44) ---
  Sol: MEQPSLSKFTRYAEHVNQELNSYCHFDAIRPGFKGVTANEGYQL
str01: M-----------A------L-SYC------P--KG-T-------
str02: M-Q-S-S------------LN------AI-P----V--------
str03: M--P-LS----Y-----Q------HF---R---K----------
str04: ME-----------EHVN-EL----H-D-----------------
str05: M---S-----------N--------FDAIR-------A-----L
str06: M-------F-R-----NQ--NS-------R--------N-G---
str07: M-------F--YA-H------------A----F-G-----GY--
str08: M---S--KFTR------------------RP----------YQ-
str09: M---S---F------V-----------A---G---VTA----Q-
str10: ME--SL---------V--------------PGF-----NE----

solution is feasible: True
solution is optimal: False
bset bound: 0


In [ ]:
bench(scsp.example.load("protein_n050k050.txt"))

--- Condition (with 20 chars) ---
str01: MRHLNIDIETYSSNDIKNGVYKYADAEDFEILLFAYSIDGGEVECLDLTR
str02: MERRAHRTHQNWDATKPRERRKQTQHRLTHPDDSIYPRIEKAEGRKEDHG
str03: MEPGAFSTALFDALCDDILHRRLESQLRFGGVQIPPEVSDPRVYAGYALL
str04: MGKFYYSNRRLAVFAQAQSRHLGGSYEQWLACVSGDSAFRAEVKARVQKD
str05: FFRENLAFQQGKAREFPSEEARANSPTSRELWVRRGGNPLSEAGAERRGT
str06: MDPSLTQVWAVEGSVLSAAVDTAETNDTEPDEGLSAENEGETRIIRITGS
str07: MAFDFSVTGNTKLDTSGFTQGVSSMTVAAGTLIADLVKTASSQLTNLAQS
str08: MAVILPSTYTDGTAACTNGSPDVVGTGTMWVNTILPGDFFWTPSGESVRV
str09: MNTGIIDLFDNHVDSIPTILPHQLATLDYLVRTIIDENRSVLLFHIMGSG
str10: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MDSKETILIEIIPKIKSYLLDTNISPKSYNDFISRNKNIFVINLYNVSTI
str12: MLLSGKKKMLLDNYETAAARGRGGDERRRGWAFDRPAIVTKRDKSDRMAH
str13: MNGEEDDNEQAAAEQQTKKAKREKPKQARKVTSEAWEHFDATDDGAECKH
str14: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str15: MRYIVSPQLVLQVGKGQEVERALYLTPYDYIDEKSPIYYFLRSHLNIQRP
str16: MPRVPVYDSPQVSPNTVPQARLATPSFATPTFRGADAPAFQDTANQQARQ
str17: MFVFLVLLPLVSSQCVNLRTRTQLPLAYTNS

-------------------------------------------------------------------------R---G-------------------G-------------------------------------------------------N-P----L---------S---E-------------A--------GAE-R-------------R---G--------------------T----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str06: M---D--PSL--------------------------TQV------------------------------------------------------------------------------------------------------------------------W----A---------------------------------------------------------V--------E-G----SV---------------------------L---------S--------A--A----VD-----------------------------------------------------------------------------------T-----A-----E---------------------T-------------N-D----------------------T-----E------------------

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------
str11: M---D---S---K------E----------------T--IL---------------IEI-I--------------P-KIK----------------------------------S----------------------------Y----------L--L-------DT---------------------------------------N---I---------S------------------------------------------------P-K-----SY-N--D--------F-----------I-----------------------------------------------------------------------S---------------------------------------------------------------R------------N------------------------------------K-----------------------------N--IF------------------VI--------------N----------------------------L----------------------------------Y----------N-V-----------S--------T--------I--------------------------------------------------------------------------------------------------------------------------------------------------------------

------------------------------------------------------V-----P-------------------------QA-------------R---------LA------------------------T-----------------------------------------P-----SFAT---------------------P-----T----F----------------------------------R---GADA-----------------------P-A-FQD---------------------------------T--A--------NQ--------------------------QARQ------------------------------------------------------------------------------------------------------------------------
str17: M---------------F-V---F-----------------LVL--L-----------------------------P----L-----------------------------------------------------------------------V--S----SQCV------------------------------------------N----LR-----T------R---T------QL-------------------------------P-----LA-Y------T-----------------------------------------N------------------------------------------------S-----------------------F------------T--------------------------R---GVY------------------------------------------------------

-------------------------------------------------------------------------------------------------------------------------------------------------------------------W-----------K-----L---------T---G------LPLPDECDFDHLI---------------------------------------------------------------------------------------------
str21: M----E--SL--------V-PGFN----E--------------K----T------H------------------------------------------------------------------------------------------------VQLS-LP----V----LQ----------------------------------------------------V--RDV--L--------V-----------------------------------------------------------------------------------------------------------R-GFGD-----------------------S--------------------------------------------------------------------V-EE------VLSE---VR-------------Q--HL--------KD-GT---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

...

-------S-G-------PT--EV---------------S-------------T-CFG--------------------S-----------------------------L-HP----F---------------------Q--------------------------------------S-L------------------------------------------K---------P-------------V------------------------------------------------------------------------MAN--------A-----LG------V-LE-----------------G----------------------K-M-FC-----------SI-------GG--------------------------------RS-------L---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str33: MA-T-----LL-------------R-S-------------L---------A--------------LF-K-----------------------------R----------NKDKP--------------------------------------------P----------------------------I------------------------------T-S------------G----S-------G--GAI----------------------------------------------------------

------------SRKHRN-------------------------------------------------------------------------------------
str37: M--------L---N----------R---------I--Q----------T----------------L-------MK-------------------------------------------------------------------------------------------TA--------------------------------------N------N----------------------------------------------------------------Y-----ET------------------IEIL---------------------------------------R------N-Y-L---------------------------------------------------------------------------------R---------------L----------------------------------------------------YII----LA-------------------------------R---------N-EE-G---R----G-------I------L-----I----------------------------Y-DD-------NI-D----------S----------------------------------------------------------------------V------------------------------------------------------------------------------------------------------------------------------------------------------------------
str38:

---------------------
str41: M------------N---------N-------------Q------------------------------------------------------------R------KK-----------------------------------------------------------TA--RP-----------------------------------SF----NMLK--------R--------------A----------------------------------------------------------------------------------------------------------R------N-------------------------------------------------------------------------------------R----V-----------S---TV-------------SQ---LA-------K-R----F---------------------------------------S----K---------------------G-----------------------L---------L-----S--------G--------------------------QG-PMKL------------------------------------------------------------------------V-------MAF--------------------------------------------------------------------------------------------------------------------------------------------------------
str42: M-------S----N--FD---------A------I----------------------------------------------

------------VF--------F----L------FV---------------------L-------------W----------------------------------KA--L-------------------------------------------------------S-L-P----------------------------------------------------------------V-----PT----------R----C--------Q------I--D------------------------------MA-----KKL---------SA-------------------------G-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str46: MA------SLL-K-------------S-------------L-------T----------------LF-K-----------------------------R-------------------------------------------------------------------T---R----D------------------------------------------------------------Q--------------------------------PP----LAS---------------G--------------------------

------------------------------------------------------------------------------------------------------------------------------------------------------------------------

solution is feasible: True
solution is optimal: False
bset bound: 0


In [ ]:
bench0(scsp.example.load("protein_n050k050.txt"))

--- Condition (with 20 chars) ---
str01: MRHLNIDIETYSSNDIKNGVYKYADAEDFEILLFAYSIDGGEVECLDLTR
str02: MERRAHRTHQNWDATKPRERRKQTQHRLTHPDDSIYPRIEKAEGRKEDHG
str03: MEPGAFSTALFDALCDDILHRRLESQLRFGGVQIPPEVSDPRVYAGYALL
str04: MGKFYYSNRRLAVFAQAQSRHLGGSYEQWLACVSGDSAFRAEVKARVQKD
str05: FFRENLAFQQGKAREFPSEEARANSPTSRELWVRRGGNPLSEAGAERRGT
str06: MDPSLTQVWAVEGSVLSAAVDTAETNDTEPDEGLSAENEGETRIIRITGS
str07: MAFDFSVTGNTKLDTSGFTQGVSSMTVAAGTLIADLVKTASSQLTNLAQS
str08: MAVILPSTYTDGTAACTNGSPDVVGTGTMWVNTILPGDFFWTPSGESVRV
str09: MNTGIIDLFDNHVDSIPTILPHQLATLDYLVRTIIDENRSVLLFHIMGSG
str10: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MDSKETILIEIIPKIKSYLLDTNISPKSYNDFISRNKNIFVINLYNVSTI
str12: MLLSGKKKMLLDNYETAAARGRGGDERRRGWAFDRPAIVTKRDKSDRMAH
str13: MNGEEDDNEQAAAEQQTKKAKREKPKQARKVTSEAWEHFDATDDGAECKH
str14: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str15: MRYIVSPQLVLQVGKGQEVERALYLTPYDYIDEKSPIYYFLRSHLNIQRP
str16: MPRVPVYDSPQVSPNTVPQARLATPSFATPTFRGADAPAFQDTANQQARQ
str17: MFVFLVLLPLVSSQCVNLRTRTQLPLAYTNS

F---------F------------------------WT-P------------SG---E------------------S-------------V---R---V-------------------------------------
str09: M------------------N----------T-------------------G----I------------------------I----D-----L-----F-----D-------------N------------------H---------V--D-S--I--------P------T------------------ILP----------H------------QL------A---------T------------L-D-----Y---L-----VR-------T-I------------I-----D-------E--N-R--------S----V-------L--------L------------------------F-------------------HI---------------M--G----------SG----------------------------------------------------------------------------------
str10: M-FVFL--VLLPLVSSQCVN--------L---------R--TR------T------------QL--------P-----P-------AYT------NSFTRG-V-----Y------------------Y--------------------PDK--V----------F----R----S------------S-----V-------LH--------S-------------------------------------------------------------------------------------------------------------------------------------------

-----------------Q--A------R----LA----------T------P-S---------FA-----T--------P---T-------------FR------G-----AD--------------A-------------P------A-----------F-----Q--D-------T-------A-------N----Q---------------------QA-R----------------------Q-------------------------------------------------------------------------------------------------------------------------------------------
str17: M-FVFL--VLLPLVSSQCVN--------L---------R--TR------T------------QL--------PLA------------YT------NSFTRG-V-----Y------------------Y--------------------PDK--V----------F----R----S------------S-----V-------LH--------S----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str18: M-FVF------------------F--------V-L-LP------L--VS-----------S-Q-------------------------------------

------------------------------------------------------
str24: M-----------------------KN--------------I-----------A-E---------------F-------------------K----------------------K------AP-E----L---AEK--L------L-------EV----------FS-N---LK------G------------N----S------------RS----L-------D-----P------MRA--GKH---D--------V------V----V-----I----------------ES-----T--------K----------KL---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str25: M----------P----Q----P------LK---Q-SL--------------------D----Q--------------S------------K---W---------------------L-R----EA--------EK-HL-R-----A---------L----------E-------S------L---V--D--------S--NL----------E----------------------------E-----------------E-------K---L----KPQ------------L-S---------------------M---------------G-------E------D---V-------QS------------------------------------------------------------------------

------YPDKVFRS--S---V-LH---------L--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str32: M------------------------------H-Q------IT-V---VS-G----------P--T----------------E-------V------S-T------------C-------------F----GS-----L---------------------H---PF---------------Q------S--L---K--------P----V----------------------------M-A----------------N---A----------L-------G----------VLE--GK------------------M--------F-CS--IG--------G-----------------------R-S----L------------------------------------------------------------------------------------------------------------------------------
str33: MA----------------------------T---L-L-R---------S----L--------------------A--------L-------------F---------------K----R-------N-------KD--------------K------------P----------------------

---------E------D---------L--DG---------F-------RD----------------F----I---R-----A--------------------H---L-------------I-----------------CL--AVD-------------T--E-----T-----------------------T------GL-----D----IY------------------------------------------------------
str44: M-FVFL--VLLPLVSSQCV----------------------------------------------------MPL--------F-N------L---------I-----T-----------T------N--Q-S------------------------------Y-------T--NSF------TRGVY-----------------YPDKVFRS--S---V-LH------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str45: M------S----------------K--DL---V------A--R---Q-----AL-----------------M-------T------A------------R--------------------------------------M-----------K---------A-------D------F-V-------------------------------F----------

solution is feasible: True
solution is optimal: False
bset bound: 0


元の `WMM_HEXALY` よりもだいぶ悪い結果となった.